In [133]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd

In [243]:
response = requests.get('https://attack.mitre.org/techniques/enterprise/')
response = requests.get('https://attack.mitre.org/tactics/TA0001/')

In [244]:
text = BeautifulSoup(response.content, 'lxml').get_text()
text = [re.sub('\s+',' ',x) for x in text.split('\n') if re.sub('\s+','',x)]
text = ' '.join(text)

In [245]:
ID_pattern = re.compile('T\d+')
sub_ID_pattern = re.compile('[^\d]\.\d+')
IDs = [start_pattern.start() for start_pattern in ID_pattern.finditer(text)]

In [246]:
techniques = {}
for i in range(len(IDs)-1):
    t = text[IDs[i]:IDs[i+1]].split('  ')[1:3]
    techniques[t[0]] = [t[1].strip()]
t = text[IDs[i+1]:].split('  ')[1:3]
techniques[t[0]] = [t[1].strip()]

In [247]:
techniques = {}
for i in range(len(IDs)-1):
    txt = text[IDs[i]:IDs[i+1]].split('  ')
    if (len(txt)<5):
        techniques[txt[1].strip()] = [txt[2].strip()]
        continue
    sub_techniques = {}
    for j in range((len(txt)-1)//3):
        sub_techniques[txt[j*3+1].strip()] = [txt[j*3+2].strip()]
    techniques[txt[1].strip()] = sub_techniques
txt = text[IDs[i+1]:].split('  ')
if (len(txt)<5):
    techniques[txt[2].strip()] = [txt[3].strip()]
else:
    sub_techniques = {}
    for j in range((len(txt)-1)//3):
        sub_techniques[txt[j*3+1].strip()] = [txt[j*3+2].strip()]
    techniques[txt[1]] = sub_techniques

In [248]:
techniques_df = []
for technique in techniques.keys():
    try:
        idx = pd.MultiIndex.from_product([[technique],techniques[technique].keys()], names = ['ID', 'sub_ID'])
    except:
        idx = pd.MultiIndex.from_product([[technique],[technique]])
    df = pd.DataFrame(techniques[technique], index = ['Description']).T
    df.index = idx
    techniques_df.append(df)
techniques_df = pd.concat(techniques_df)
techniques_df

Description
Drive-by Compromise                 Drive-by Compromise                                 Adversaries may gain access to a system throug...
Exploit Public-Facing Application   Exploit Public-Facing Application                   Adversaries may attempt to take advantage of a...
External Remote Services            External Remote Services                            Adversaries may leverage external-facing remot...
Hardware Additions                  Hardware Additions                                  Adversaries may introduce computer accessories...
Phishing                            Phishing                                            Adversaries may send phishing messages to gain...
                                    Spearphishing Attachment                            Adversaries may send spearphishing emails with...
                                    Spearphishing Link                                  Adversaries may send spearphishing emails with...
                                    Spearphishing via Service                           Adversaries may send spearphishing messages vi...
Replication Through Removable Media Replication Through Removable Media                 Adversaries may move onto systems, possibly th...
Supply Chain Compromise             Supply Chain Compromise                             Adversaries may manipulate products or product...
                                    Compromise Software Dependencies and Developmen...  Adversaries may manipulate software dependenci...
                                    Compromise Software Supply Chain                    Adversaries may manipulate application softwar...
                                    Compromise Hardware Supply Chain                    Adversaries may manipulate hardware components...
Trusted Relationship                Trusted Relationship                                Adversaries may breach or otherwise leverage o...
 Valid Accounts                     Valid Accounts                                      Adversaries may obtain and abuse credentials o...
                                    Default Accounts                                    Adversaries may obtain and abuse credentials o...
                                    Domain Accounts                                     Adversaries may obtain and abuse credentials o...
                                    Local Accounts                                      Adversaries may obtain and abuse credentials o...
                                    Cloud Accounts                                      Adversaries may obtain and abuse credentials o...